In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import datetime
import pandas as pd


class udn_news:
    
    def __init__(self, keyword, result = []):
        
        self.keyword = keyword
        self.result = result
    
    def get_udn_news(self, date):


        starttime = datetime.datetime.now()
        print(starttime)
        
        url = "https://money.udn.com/search/result/1001/{}".format(self.keyword)

        headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
        "cache-control": "max-age=0",
        "referer": "https://money.udn.com/search/result/1001/{}/1".format(self.keyword).encode('utf-8'),
        "sec-ch-ua-mobile": "?0",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"

        }
        while True:

            res = requests.get(url,headers = headers)

            soup = BeautifulSoup(res.text, "html.parser")

            for dt in soup.select("div#search_content dl dt"):

                try:                             #防止文章遺失造成程式停止
                    each_article_text = ""                                 #建立空字串方便後面加入文章內容
                    article_date = dt.select_one("a span.cat").text[5:]  #取得文章日期
                    article_url = dt.select_one("a")["href"]               #取得文章url
                    title = dt.select_one("a h3").text                     #取得文章標題


                    print(article_date)
                    print(title)
                    print(article_url)


                    #設定時間區間
                    article_date_str = "".join(article_date.split("/"))
                    if int(article_date_str) < int(date):
                        break

                    else:

                        headers["referer"] = url.encode('utf-8')
                        res2 = requests.get(article_url, headers = headers)
                        soup2 = BeautifulSoup(res2.text, "html.parser")

                        each_article = {}  #建立空字典方便接下來暫存統整每篇文章內容
                        
                        article_hour = soup2.select_one("div.shareBar__info--author").text.split(" ")[1][:5]
                        print(article_hour)


                        paragraphs = soup2.select("div#article_body p")

                        for p in paragraphs:
                            if p.text.strip() != "":
                                each_article_text += p.text.strip()
                            else:
                                pass

                        print(each_article_text)


                        #加入關鍵字、標題、日期、時間、網址、內容

                        each_article["關鍵字"] = self.keyword
                        each_article["標題"] = title
                        each_article["日期"] = article_date
                        each_article["時間"] = article_hour
                        each_article["網址"] = article_url
                        each_article["內容"] = each_article_text


                        #將每一篇文章資訊加入list
                        self.result.append(each_article)
                        
                        #以防萬一邊爬邊存 
                        df_new = pd.DataFrame.from_records(self.result)
                        df_new.to_csv("new_udn_covid19_news.csv", encoding = "utf-8-sig", index = False)


                        #設定隨機休息時間
                        sleep_time = random.randint(3,10)
                        print("sleep time: %s sec"%(sleep_time))
                        time.sleep(sleep_time)

                except:
                    pass

            try:
                url = "https://money.udn.com" + soup.select("gonext a")[0]['href'] 
                if int(article_date_str) < int(date):
                        break

                else:
                    continue

            except:
                break
                
        endtime = datetime.datetime.now()
        print(endtime)
        
        return "finish"
    
    def get_content(self):
        df = pd.DataFrame.from_records(self.result)
        df.to_csv("udn_covid19_news.csv", encoding = "utf-8-sig", index = False)
        #columns = [關鍵字,標題,日期,時間,網址,內容] 
        
        return self.result
    
    def append_content(self):
        df = pd.DataFrame.from_records(self.result)
        df.to_csv("udn_covid19_news.csv", mode='a', encoding = "utf-8-sig", index = False)
        
        return self.result


In [ ]:
A = udn_news("新冠肺炎")
A.get_udn_news("20191201")

In [ ]:
abc = A.get_content()

In [ ]:
b =A.append_content()